In [ ]:
import requests
import json
import pandas as pd
from pprint import pprint
from time import sleep
import os

from spotify_credentials import client_id, client_secret


In [ ]:
#getting spotify access token
url = "https://accounts.spotify.com/api/token"
header = {"Content-Type": "application/x-www-form-urlencoded"}
data = {"grant_type":"client_credentials",
        "client_id" : client_id,
        "client_secret" : client_secret}

r = requests.post(url, data=data, headers=header).json()

spotify_api_key = r['access_token']


In [ ]:
songdf = pd.read_csv(r"C:\Users\kevin\bootcamp\Project_1\resources\top100_allYears_withID_final.csv")
songdf["duration"] = 0
songdf["loudness"] = 0
songdf["tempo"] = 0
songdf["time_signiture"] = 0
songdf["key"] = 0
songdf = songdf.iloc[:,[5,1,2,3,4,6,7,8,9,10]]
#songdf = songdf.head(1)

In [ ]:
spotify_analysis_endpoint = "https://api.spotify.com/v1/audio-analysis/"
spotify_headers = {'Authorization': 'Bearer {}'.format(spotify_api_key)}

for index, row in songdf.iterrows():

    song_id = str(songdf.iat[index,0])

    request_url = spotify_analysis_endpoint + song_id

    try:
        spotify_response = requests.get(request_url, headers = spotify_headers).json()
        songdf.iat[index,5] = float(spotify_response["track"]["duration"])
        songdf.iat[index,6] = float(spotify_response["track"]["loudness"])
        songdf.iat[index,7] = float(spotify_response["track"]["tempo"])
        songdf.iat[index,8] = float(spotify_response["track"]["time_signature"])
        songdf.iat[index,9] = float(spotify_response["track"]["key"])
        sleep(0.5)
        
    except:
        if spotify_response == {'error': {'status': 429}}:
            print(f"{spotify_response} on song ID = {song_id}")
        else:
            print(f"Failure on ID: {song_id}")
        pass
        


songdf.head()

path = r"C:\Users\kevin\bootcamp\Project_1\resources"
songdf.to_csv(os.path.join(path,f"master_data.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas

In [ ]:
songdf = pd.read_csv(r"C:\Users\kevin\bootcamp\Project_1\resources\master_data.csv")
songdf = songdf.iloc[:,[1,2,3,4,5,6,7,8,9,10]]
#songdf = songdf.head(2010)

In [ ]:
#WARNING!!! MAKE SURE TO READ IN CORRECT CSV TO NOT OVERWRITE PAST CALLS!!!!!!!!
#this code block mirrors the function above, calling on the Spotify analysis endpoint. 
#the primary difference is that it pickups up at a point in the read in csv, where data has not been written from previous calls.
#this code was created to deal with the call limits reached with spotify's api.

spotify_analysis_endpoint = "https://api.spotify.com/v1/audio-analysis/"
spotify_headers = {'Authorization': 'Bearer {}'.format(spotify_api_key)}

for index, row in songdf.iterrows():

    #nesting the main code block inside an conditional, which should help pickin up  
    if songdf.iat[index,5] != 0:
        pass
    
    elif songdf.iat[index,5] == 0:
        song_id = str(songdf.iat[index,0])
    
        request_url = spotify_analysis_endpoint + song_id
    
        try:
            spotify_response = requests.get(request_url, headers = spotify_headers).json()
            songdf.iat[index,5] = float(spotify_response["track"]["duration"])
            songdf.iat[index,6] = float(spotify_response["track"]["loudness"])
            songdf.iat[index,7] = float(spotify_response["track"]["tempo"])
            songdf.iat[index,8] = float(spotify_response["track"]["time_signature"])
            songdf.iat[index,9] = float(spotify_response["track"]["key"])
            sleep(0.5)
            
        except:
            if spotify_response == {'error': {'status': 429}}:
                print(f"{spotify_response} on song ID = {song_id}")
            else:
                print(f"Failure on ID: {song_id}")
            pass

    else:
        print("pickup point error")
        


songdf.head()

path = r"C:\Users\kevin\bootcamp\Project_1\resources"
songdf.to_csv(os.path.join(path,f"master_data_fullList.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas